In [1]:
import json
import pandas as pd

In [2]:
import os
import glob

# Base directory containing all folders
base_directory = "./generated_answers/gemini-1.5-pro-latest/test_must_why"

# Use glob to find all JSONL files recursively
jsonl_files = glob.glob(os.path.join(base_directory, "**", "*.jsonl"), recursive=True)

# Example: Print all found JSONL file paths
#print(jsonl_files)

In [3]:
# List of JSONL files
files = [['dependent_real_after_basic_binary_explain.jsonl', 'dependent_real_before_basic_binary_explain.jsonl'], ['nondependent_real_after_basic_binary_explain_nl.jsonl', 'nondependent_real_before_basic_binary_explain_nl.jsonl']]
files_to_use = files[1]

In [4]:
#Filter the list of found files based on your selection
selected_files = [file for file in jsonl_files if os.path.basename(file) in files_to_use]

# Example: Print selected file paths
print(selected_files)

['./generated_answers/gemini-1.5-pro-latest/test_must_why/nondependent_real_after_basic_binary_explain_nl.jsonl', './generated_answers/gemini-1.5-pro-latest/test_must_why/nondependent_real_before_basic_binary_explain_nl.jsonl']


In [5]:
data = []

for file_path in selected_files:
    with open(file_path, 'r') as f:
        for line in f:
            data.append(json.loads(line))

# Convert the collected data into a DataFrame
df = pd.DataFrame(data)

In [6]:
df

,plan_idx,title,question_idx,steps,question_type,step_pair_idx_asked_about,binary_question,why_question,model_name,prompt_key,model_input,model_answer
0,710,spiced-guava-cake,710,"[Preheat oven to 180 C / Gas 4., Grease and fl...",nondependent_real_after,"[3, 4]",Must Step 5 happen after Step 4?,Why is it not necessary for Step 5 to happen a...,gemini-1.5-pro-latest,basic_binary_explain_nl,"Given a goal, a procedure to achieve that goal...",Answer 1: no\nAnswer 2: The order of steps 4 a...
1,711,chocolate-chocolate-cake,711,[Preheat the oven to 180 degrees C / Gas Mark ...,nondependent_real_after,"[1, 5]",Must Step 6 happen after Step 2?,Why is it not necessary for Step 6 to happen a...,gemini-1.5-pro-latest,basic_binary_explain_nl,"Given a goal, a procedure to achieve that goal...",Answer 1: no\nAnswer 2: Steps 2-5 are about pr...
2,712,chicken-prawn-and-rice-soup,712,"[Mix together the prawns, chicken, egg and cor...",nondependent_real_after,"[3, 10]",Must Step 11 happen after Step 4?,Why is it not necessary for Step 11 to happen ...,gemini-1.5-pro-latest,basic_binary_explain_nl,"Given a goal, a procedure to achieve that goal...",Answer 1: no\nAnswer 2: Steps 11 and 4 can be ...
3,713,green-tofu-curry,713,"[Place water in a medium saucepan, and stir in...",nondependent_real_after,"[0, 4]",Must Step 5 happen after Step 1?,Why is it not necessary for Step 5 to happen a...,gemini-1.5-pro-latest,basic_binary_explain_nl,"Given a goal, a procedure to achieve that goal...",Answer 1: no\nAnswer 2: Steps 1-4 and Steps 5-...
4,714,very-vanilla-ry-cheesecake,714,[Preheat oven to 150 C / Gas 2 and lightly but...,nondependent_real_after,"[4, 7]",Must Step 8 happen after Step 5?,Why is it not necessary for Step 8 to happen a...,gemini-1.5-pro-latest,basic_binary_explain_nl,"Given a goal, a procedure to achieve that goal...",Answer 1: yes\nAnswer 2: The cheesecake fillin...
...,...,...,...,...,...,...,...,...,...,...,...,...
1415,1415,chocolate-chocolate-cake,1415,[Preheat the oven to 180 degrees C / Gas Mark ...,nondependent_real_before,"[2, 18]",Must Step 3 happen before Step 19?,Why is it not necessary for Step 3 to happen b...,gemini-1.5-pro-latest,basic_binary_explain_nl,"Given a goal, a procedure to achieve that goal...",Answer 1: yes\nAnswer 2: The chocolate mixture...
1416,1416,chocolate-chocolate-cake,1416,[Preheat the oven to 180 degrees C / Gas Mark ...,nondependent_real_before,"[5, 18]",Must Step 6 happen before Step 19?,Why is it not necessary for Step 6 to happen b...,gemini-1.5-pro-latest,basic_binary_explain_nl,"Given a goal, a procedure to achieve that goal...",Answer 1: yes\nAnswer 2: Step 6 outlines the s...
1417,1417,chicken and vegetable pie,1417,"[Preheat oven to 220 C / Gas mark 7., In a sau...",nondependent_real_before,"[3, 5]",Must Step 4 happen before Step 6?,Why is it not necessary for Step 4 to happen b...,gemini-1.5-pro-latest,basic_binary_explain_nl,"Given a goal, a procedure to achieve that goal...",Answer 1: yes\nAnswer 2: The chicken and veget...
1418,1418,chocolate-chocolate-cake,1418,[Preheat the oven to 180 degrees C / Gas Mark ...,nondependent_real_before,"[10, 14]",Must Step 11 happen before Step 15?,Why is it not necessary for Step 11 to happen ...,gemini-1.5-pro-latest,basic_binary_explain_nl,"Given a goal, a procedure to achieve that goal...",Answer 1: no\nAnswer 2: The egg whites can be ...


In [20]:
df.loc[1, 'model_answer'].split("\n", 1)[1].replace('Answer 2: ', '').replace('\n', '')

'Steps 2-5 are about preparing the chocolate mixture while steps 6-8 are about creaming butter and sugar for the cake batter, both can be done independently. '

In [7]:
import ast

In [21]:
def clean_model_answer(text):
    try:
        # Remove newline characters and extra backslashes
        #text = text.replace('\n', '').replace('\\', '').replace("json", "").strip("```")
        # Ensure the JSON string is properly closed
        #if not text.endswith('}'):
        #    text += '}'
        # Parse the string as JSON
        #data = json.loads(text)
        # Extract the 'binary_answer' and 'why_answer'
        data = text.split("\n", 1)
        binary_answer = data[0].replace('Answer 1: ', '')
        why_answer = data[1].replace('Answer 2: ', '').replace('\n', '')
        return pd.Series([binary_answer, why_answer])
    except json.JSONDecodeError as e:
        print(f"JSON parsing error: {e}")
        print(f"Problematic text: {text}")
        return pd.Series(['', ''])
    except Exception as e:
        print(f"Unexpected error: {e}")
        print(f"Problematic text: {text}")
        return pd.Series(['', ''])

In [22]:
# Apply the cleaning function to the 'model_answer' column
df[['binary_answer', 'why_answer']] = df['model_answer'].apply(clean_model_answer)

In [23]:
df

,plan_idx,title,question_idx,steps,question_type,step_pair_idx_asked_about,binary_question,why_question,model_name,prompt_key,model_input,model_answer,binary_answer,why_answer
0,710,spiced-guava-cake,710,"[Preheat oven to 180 C / Gas 4., Grease and fl...",nondependent_real_after,"[3, 4]",Must Step 5 happen after Step 4?,Why is it not necessary for Step 5 to happen a...,gemini-1.5-pro-latest,basic_binary_explain_nl,"Given a goal, a procedure to achieve that goal...",Answer 1: no\nAnswer 2: The order of steps 4 a...,no,The order of steps 4 and 5 doesn't impact the ...
1,711,chocolate-chocolate-cake,711,[Preheat the oven to 180 degrees C / Gas Mark ...,nondependent_real_after,"[1, 5]",Must Step 6 happen after Step 2?,Why is it not necessary for Step 6 to happen a...,gemini-1.5-pro-latest,basic_binary_explain_nl,"Given a goal, a procedure to achieve that goal...",Answer 1: no\nAnswer 2: Steps 2-5 are about pr...,no,Steps 2-5 are about preparing the chocolate mi...
2,712,chicken-prawn-and-rice-soup,712,"[Mix together the prawns, chicken, egg and cor...",nondependent_real_after,"[3, 10]",Must Step 11 happen after Step 4?,Why is it not necessary for Step 11 to happen ...,gemini-1.5-pro-latest,basic_binary_explain_nl,"Given a goal, a procedure to achieve that goal...",Answer 1: no\nAnswer 2: Steps 11 and 4 can be ...,no,Steps 11 and 4 can be done in any order becaus...
3,713,green-tofu-curry,713,"[Place water in a medium saucepan, and stir in...",nondependent_real_after,"[0, 4]",Must Step 5 happen after Step 1?,Why is it not necessary for Step 5 to happen a...,gemini-1.5-pro-latest,basic_binary_explain_nl,"Given a goal, a procedure to achieve that goal...",Answer 1: no\nAnswer 2: Steps 1-4 and Steps 5-...,no,Steps 1-4 and Steps 5-8 can be performed indep...
4,714,very-vanilla-ry-cheesecake,714,[Preheat oven to 150 C / Gas 2 and lightly but...,nondependent_real_after,"[4, 7]",Must Step 8 happen after Step 5?,Why is it not necessary for Step 8 to happen a...,gemini-1.5-pro-latest,basic_binary_explain_nl,"Given a goal, a procedure to achieve that goal...",Answer 1: yes\nAnswer 2: The cheesecake fillin...,yes,The cheesecake filling must be added on top of...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415,1415,chocolate-chocolate-cake,1415,[Preheat the oven to 180 degrees C / Gas Mark ...,nondependent_real_before,"[2, 18]",Must Step 3 happen before Step 19?,Why is it not necessary for Step 3 to happen b...,gemini-1.5-pro-latest,basic_binary_explain_nl,"Given a goal, a procedure to achieve that goal...",Answer 1: yes\nAnswer 2: The chocolate mixture...,yes,The chocolate mixture needs to be made before ...
1416,1416,chocolate-chocolate-cake,1416,[Preheat the oven to 180 degrees C / Gas Mark ...,nondependent_real_before,"[5, 18]",Must Step 6 happen before Step 19?,Why is it not necessary for Step 6 to happen b...,gemini-1.5-pro-latest,basic_binary_explain_nl,"Given a goal, a procedure to achieve that goal...",Answer 1: yes\nAnswer 2: Step 6 outlines the s...,yes,Step 6 outlines the start of mixing the cake b...
1417,1417,chicken and vegetable pie,1417,"[Preheat oven to 220 C / Gas mark 7., In a sau...",nondependent_real_before,"[3, 5]",Must Step 4 happen before Step 6?,Why is it not necessary for Step 4 to happen b...,gemini-1.5-pro-latest,basic_binary_explain_nl,"Given a goal, a procedure to achieve that goal...",Answer 1: yes\nAnswer 2: The chicken and veget...,yes,The chicken and vegetables need to be cooked a...
1418,1418,chocolate-chocolate-cake,1418,[Preheat the oven to 180 degrees C / Gas Mark ...,nondependent_real_before,"[10, 14]",Must Step 11 happen before Step 15?,Why is it not necessary for Step 11 to happen ...,gemini-1.5-pro-latest,basic_binary_explain_nl,"Given a goal, a procedure to achieve that goal...",Answer 1: no\nAnswer 2: The egg whites can be ...,no,The egg whites can be prepared while the cake ...


In [24]:
df_copy = df.copy()

In [25]:
df_copy = df_copy.drop('model_answer', axis=1)

In [26]:
df_copy

,plan_idx,title,question_idx,steps,question_type,step_pair_idx_asked_about,binary_question,why_question,model_name,prompt_key,model_input,binary_answer,why_answer
0,710,spiced-guava-cake,710,"[Preheat oven to 180 C / Gas 4., Grease and fl...",nondependent_real_after,"[3, 4]",Must Step 5 happen after Step 4?,Why is it not necessary for Step 5 to happen a...,gemini-1.5-pro-latest,basic_binary_explain_nl,"Given a goal, a procedure to achieve that goal...",no,The order of steps 4 and 5 doesn't impact the ...
1,711,chocolate-chocolate-cake,711,[Preheat the oven to 180 degrees C / Gas Mark ...,nondependent_real_after,"[1, 5]",Must Step 6 happen after Step 2?,Why is it not necessary for Step 6 to happen a...,gemini-1.5-pro-latest,basic_binary_explain_nl,"Given a goal, a procedure to achieve that goal...",no,Steps 2-5 are about preparing the chocolate mi...
2,712,chicken-prawn-and-rice-soup,712,"[Mix together the prawns, chicken, egg and cor...",nondependent_real_after,"[3, 10]",Must Step 11 happen after Step 4?,Why is it not necessary for Step 11 to happen ...,gemini-1.5-pro-latest,basic_binary_explain_nl,"Given a goal, a procedure to achieve that goal...",no,Steps 11 and 4 can be done in any order becaus...
3,713,green-tofu-curry,713,"[Place water in a medium saucepan, and stir in...",nondependent_real_after,"[0, 4]",Must Step 5 happen after Step 1?,Why is it not necessary for Step 5 to happen a...,gemini-1.5-pro-latest,basic_binary_explain_nl,"Given a goal, a procedure to achieve that goal...",no,Steps 1-4 and Steps 5-8 can be performed indep...
4,714,very-vanilla-ry-cheesecake,714,[Preheat oven to 150 C / Gas 2 and lightly but...,nondependent_real_after,"[4, 7]",Must Step 8 happen after Step 5?,Why is it not necessary for Step 8 to happen a...,gemini-1.5-pro-latest,basic_binary_explain_nl,"Given a goal, a procedure to achieve that goal...",yes,The cheesecake filling must be added on top of...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415,1415,chocolate-chocolate-cake,1415,[Preheat the oven to 180 degrees C / Gas Mark ...,nondependent_real_before,"[2, 18]",Must Step 3 happen before Step 19?,Why is it not necessary for Step 3 to happen b...,gemini-1.5-pro-latest,basic_binary_explain_nl,"Given a goal, a procedure to achieve that goal...",yes,The chocolate mixture needs to be made before ...
1416,1416,chocolate-chocolate-cake,1416,[Preheat the oven to 180 degrees C / Gas Mark ...,nondependent_real_before,"[5, 18]",Must Step 6 happen before Step 19?,Why is it not necessary for Step 6 to happen b...,gemini-1.5-pro-latest,basic_binary_explain_nl,"Given a goal, a procedure to achieve that goal...",yes,Step 6 outlines the start of mixing the cake b...
1417,1417,chicken and vegetable pie,1417,"[Preheat oven to 220 C / Gas mark 7., In a sau...",nondependent_real_before,"[3, 5]",Must Step 4 happen before Step 6?,Why is it not necessary for Step 4 to happen b...,gemini-1.5-pro-latest,basic_binary_explain_nl,"Given a goal, a procedure to achieve that goal...",yes,The chicken and vegetables need to be cooked a...
1418,1418,chocolate-chocolate-cake,1418,[Preheat the oven to 180 degrees C / Gas Mark ...,nondependent_real_before,"[10, 14]",Must Step 11 happen before Step 15?,Why is it not necessary for Step 11 to happen ...,gemini-1.5-pro-latest,basic_binary_explain_nl,"Given a goal, a procedure to achieve that goal...",no,The egg whites can be prepared while the cake ...


In [27]:
counts = df_copy['binary_answer'].value_counts()
num_yes = counts['yes']
num_no = counts['no']

In [28]:
num_yes

645

In [29]:
num_no

731

In [30]:
df_yes = df_copy[df_copy['binary_answer'] == 'yes']

In [31]:
df_yes

,plan_idx,title,question_idx,steps,question_type,step_pair_idx_asked_about,binary_question,why_question,model_name,prompt_key,model_input,binary_answer,why_answer
4,714,very-vanilla-ry-cheesecake,714,[Preheat oven to 150 C / Gas 2 and lightly but...,nondependent_real_after,"[4, 7]",Must Step 8 happen after Step 5?,Why is it not necessary for Step 8 to happen a...,gemini-1.5-pro-latest,basic_binary_explain_nl,"Given a goal, a procedure to achieve that goal...",yes,The cheesecake filling must be added on top of...
7,717,spanish-rabbit-stew,717,"[After slicing all of your ingredients, throw ...",nondependent_real_after,"[2, 8]",Must Step 9 happen after Step 3?,Why is it not necessary for Step 9 to happen a...,gemini-1.5-pro-latest,basic_binary_explain_nl,"Given a goal, a procedure to achieve that goal...",yes,You should season the stew after it is finishe...
9,719,tangy-gammon-in-cider,719,"[Preheat the oven to 200 degrees /gas mark 6.,...",nondependent_real_after,"[0, 6]",Must Step 7 happen after Step 1?,Why is it not necessary for Step 7 to happen a...,gemini-1.5-pro-latest,basic_binary_explain_nl,"Given a goal, a procedure to achieve that goal...",yes,The oven needs to be preheated before the gamm...
10,720,lemon-zested-strawberry-shortcakes,720,"[In a small bowl, combine strawberries and sug...",nondependent_real_after,"[11, 13]",Must Step 14 happen after Step 12?,Why is it not necessary for Step 14 to happen ...,gemini-1.5-pro-latest,basic_binary_explain_nl,"Given a goal, a procedure to achieve that goal...",yes,You need to whip the cream before you can asse...
12,722,almond and apple cake,722,"[Preheat the oven to 170 C / Gas mark 3., Brus...",nondependent_real_after,"[0, 1]",Must Step 2 happen after Step 1?,Why is it not necessary for Step 2 to happen a...,gemini-1.5-pro-latest,basic_binary_explain_nl,"Given a goal, a procedure to achieve that goal...",yes,Greasing the pan while it's cold is easier and...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1412,1412,chocolate-chocolate-cake,1412,[Preheat the oven to 180 degrees C / Gas Mark ...,nondependent_real_before,"[3, 14]",Must Step 4 happen before Step 15?,Why is it not necessary for Step 4 to happen b...,gemini-1.5-pro-latest,basic_binary_explain_nl,"Given a goal, a procedure to achieve that goal...",yes,The chocolate mixture needs to be cooled befor...
1413,1413,spiced butternut squash soup,1413,"[Preheat the oven to 190 C / Gas mark 6., Pour...",nondependent_real_before,"[3, 10]",Must Step 4 happen before Step 11?,Why is it not necessary for Step 4 to happen b...,gemini-1.5-pro-latest,basic_binary_explain_nl,"Given a goal, a procedure to achieve that goal...",yes,The squash needs to be cooked before it can be...
1415,1415,chocolate-chocolate-cake,1415,[Preheat the oven to 180 degrees C / Gas Mark ...,nondependent_real_before,"[2, 18]",Must Step 3 happen before Step 19?,Why is it not necessary for Step 3 to happen b...,gemini-1.5-pro-latest,basic_binary_explain_nl,"Given a goal, a procedure to achieve that goal...",yes,The chocolate mixture needs to be made before ...
1416,1416,chocolate-chocolate-cake,1416,[Preheat the oven to 180 degrees C / Gas Mark ...,nondependent_real_before,"[5, 18]",Must Step 6 happen before Step 19?,Why is it not necessary for Step 6 to happen b...,gemini-1.5-pro-latest,basic_binary_explain_nl,"Given a goal, a procedure to achieve that goal...",yes,Step 6 outlines the start of mixing the cake b...


In [32]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans

In [139]:
why_answer = df_yes['why_answer'].tolist()

In [140]:
# Create a bag of words representation
vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(why_answer)

In [141]:
#Perform K-means clustering
kmeans = KMeans(n_clusters=4)  # Adjust the number of clusters as needed
kmeans.fit(X)

KMeans(n_clusters=4)

In [142]:
# Add cluster labels
df_no['cluster'] = kmeans.labels_

/var/folders/j9/5zmzdnn93jj4z_0j5tw59c540000gn/T/ipykernel_39006/3760491917.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no['cluster'] = kmeans.labels_


In [143]:
df_no

,plan_idx,title,question_idx,steps,question_type,step_pair_idx_asked_about,binary_question,why_question,model_name,prompt_key,model_input,binary_answer,why_answer,cluster
4,4,chocolate-chocolate-cake,4,[Preheat the oven to 180 degrees C / Gas Mark ...,dependent_real_after,"[0, 20]",Must Step 21 happen after Step 1?,Why must Step 21 happen after Step 1?,gpt-3.5-turbo-1106,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",no,"No, Step 21 does not have to happen after Step...",0
6,6,spiced butternut squash soup,6,"[Preheat the oven to 190 C / Gas mark 6., Pour...",dependent_real_after,"[1, 14]",Must Step 15 happen after Step 2?,Why must Step 15 happen after Step 2?,gpt-3.5-turbo-1106,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",no,Step 15 does not need to happen after Step 2 b...,3
7,7,spiced butternut squash soup,7,"[Preheat the oven to 190 C / Gas mark 6., Pour...",dependent_real_after,"[3, 14]",Must Step 15 happen after Step 4?,Why must Step 15 happen after Step 4?,gpt-3.5-turbo-1106,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",no,"Step 15 does not need to happen after Step 4, ...",3
8,8,vegetable-terrine,8,[Add the gelatin in a bowl of cold water until...,dependent_real_after,"[4, 10]",Must Step 11 happen after Step 5?,Why must Step 11 happen after Step 5?,gpt-3.5-turbo-1106,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",no,"Step 11 does not have to happen after Step 5, ...",3
10,10,jam-drop-biscuits,10,[Beat butter until fluffy and gradually add th...,dependent_real_after,"[1, 3]",Must Step 4 happen after Step 2?,Why must Step 4 happen after Step 2?,gpt-3.5-turbo-1106,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",no,"Step 4 can happen after Step 2, but it doesn't...",3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1394,684,peppers-with-tuna-and-mushrooms,684,[Heat 2 tbsp olive oil in a frying pan and add...,dependent_real_before,"[4, 9]",Must Step 5 happen before Step 10?,Why must Step 5 happen before Step 10?,gpt-3.5-turbo-1106,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",no,Step 5 does not need to happen before Step 10 ...,0
1399,689,vegetable-terrine,689,[Add the gelatin in a bowl of cold water until...,dependent_real_before,"[13, 14]",Must Step 14 happen before Step 15?,Why must Step 14 happen before Step 15?,gpt-3.5-turbo-1106,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",no,Step 15 must happen before Step 14 because the...,3
1404,694,chocolate-chocolate-cake,694,[Preheat the oven to 180 degrees C / Gas Mark ...,dependent_real_before,"[8, 20]",Must Step 9 happen before Step 21?,Why must Step 9 happen before Step 21?,gpt-3.5-turbo-1106,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",no,"No, Step 21 can happen before Step 9 as it is ...",3
1413,703,curried bbq mussels,703,[Preheat barbecue for medium-high heat and lig...,dependent_real_before,"[2, 10]",Must Step 3 happen before Step 11?,Why must Step 3 happen before Step 11?,gpt-3.5-turbo-1106,basic_binary_explain,"Given a goal, a procedure to achieve that goal...",no,"Step 11 can happen after Step 3, as Step 3 inv...",0


In [33]:
df_yes.to_csv('nondependent_real_basic_binary_explain_yes.csv', index=False)

In [57]:
#rows_with_label = df_yes[df_yes['cluster'] == 0]